## calculate f1 score


In [ ]:
from sklearn.metrics import f1_score
import ast


data=[]
prediction_new = []
label_new = []

# Read the file and extract the prediction and label columns
line = "17625	['O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', '[CLS]', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O']	['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-A2', 'I-A2', 'O', 'B-A1', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']"


uid, prediction, label = line.strip().split("\t")
data.append((prediction.strip("[]"), label.strip("[]")))

print(f"data: type: {type(data)}")
print(f"prediction: {prediction}")
print(f"label: {label}")
# convert list to string
predictions_data = [d[0] for d in data]
labels_data = [d[1] for d in data]

lineTok = ast.literal_eval(label)
lineLab = ast.literal_eval(prediction)

print(f"lineTok: {lineTok}")
print(f"lineLab: {lineLab}")
for (Tok, Lab) in zip(lineTok, lineLab):

    if Lab in ['[CLS]','[SEP]', 'X']: # replace non-text tokens with O. These will not be evaluated.
        prediction_new.append('O')
        label_new.append('O')
        continue
    if(Lab == "B-V"):
        prediction_new.append("V")
    else:
        prediction_new.append(Lab)
        label_new.append(Tok)

print(f"prediction_new: {prediction_new}")
print(f"label_new: {label_new}")

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import recall_score as class_recall_score

new1 = [label_new]
print(f"new1: {new1}")
new2 = [prediction_new]

result_f1 = f1_score(label_new, prediction_new,average="micro")

print(f"result_f1: {result_f1}")

data: type: <class 'list'>
prediction: ['O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', '[CLS]', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O']
label: ['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-A2', 'I-A2', 'O', 'B-A1', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']
lineTok: ['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-A2', 'I-A2', 'O', 'B-A1', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']
lineLab: ['O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', '[CLS]', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O']
prediction_new: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
label_new: ['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-A2', 'I-A2', 'O', 'O', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']
new1: [['B-A0', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-A2', 'I-A2', 'O', 'O', 'I-A1', 'I-A1', 'O', 'O', 'O', 'O']]
result_f1: 0.75


In [ ]:
data = []
labels = []
predictions = []

filePath = "./output/conllsrl_test_predictions_29.tsv"
with open(filePath, "r") as f:
    for line in f:
        # delete the first line
        if "uid" in line:
            continue
        uid, prediction, label = line.strip().split("\t")
        labels.append(label)
        predictions.append(prediction)
        data.append((prediction.strip("[]"), label.strip("[]")))

print("prediction: ", type(data))
# apply this conversion to all the elements in the list
lineToks = [ast.literal_eval(label) for label in labels]
lineLabs = [ast.literal_eval(prediction) for prediction in predictions]

trueLabels = [] # labels
predictLabels = [] # predictions

for lineTok, lineLab in zip(lineToks, lineLabs):
    if lineLab in ['[CLS]','[SEP]', 'X']: # replace non-text tokens with O. These will not be evaluated.
        predictLabels.append('O')
        trueLabels.append('O')
        continue
    if(lineLab == "B-V"):
        predictLabels.append("V")
    else:
        predictLabels.append(lineLab)
        trueLabels.append(lineTok)

# flatten the list of lists into a single list
trueLabels = [item for sublist in trueLabels for item in sublist]
predictLabels = [item for sublist in predictLabels for item in sublist]

result_f1 = f1_score(trueLabels, predictLabels,average="micro")

print(f"result_f1: {result_f1}")

----


##test original model

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertConfig

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

# dmis-lab/biobert-base-cased-v1.2

tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2', do_lower_case=True,truncation=True)

In [ ]:

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)
    outputs2 = model(tokens_tensor2, segments_tensors2)
    hidden_states = outputs[2]
    hidden_states2 = outputs2[2]
outputs.hidden_states

(tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
          [-0.2659,  0.0911, -0.2904,  ..., -0.0518,  0.2286, -0.0701],
          [-0.1257, -0.1021,  0.0349,  ..., -0.6787,  0.5009,  0.8158],
          ...,
          [ 0.7358,  0.3943,  0.0779,  ..., -0.6939, -0.3449, -0.0302],
          [ 0.0593, -0.3172,  0.0352,  ...,  0.1207, -0.4333, -0.0917],
          [-0.4895,  0.1848,  0.3578,  ..., -0.3979, -0.0418, -0.2135]]]),
 tensor([[[ 0.1778,  0.0206,  0.0094,  ..., -0.0476,  0.1028,  0.1027],
          [-0.0802,  0.1250, -0.0769,  ...,  0.0508,  0.6060, -0.4218],
          [ 0.0118,  0.0407, -0.3726,  ..., -0.9866,  0.8263,  0.6549],
          ...,
          [ 1.3002,  0.2144,  0.6183,  ..., -0.4371, -0.7979, -0.0119],
          [ 0.0541, -0.2116,  0.2216,  ...,  0.2839, -0.3027, -0.3254],
          [-0.2223,  0.1883,  0.2541,  ..., -0.5151,  0.3750, -0.1689]]]),
 tensor([[[ 0.1000, -0.2102, -0.0491,  ..., -0.0036, -0.0153,  0.0920],
          [ 0.1070,  0.0310,

In [ ]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings2 = torch.stack(hidden_states2, dim=0)

token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings2 = torch.squeeze(token_embeddings2, dim=1)
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)
token_embeddings2 = token_embeddings2.permute(1,0,2)

In [ ]:
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []
token_vecs_cat2 = []
# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

for token in token_embeddings2:
    cat_vec2 = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    token_vecs_cat2.append(cat_vec2)

## test cosine similarity

```python

In [ ]:
def cosine_similarity_2_tensors(tensor1, tensor2):

    # Compute the dot product
    dot_product = torch.dot(tensor1, tensor2)

    # Compute the L2 (Euclidean) norms
    norm_tensor1 = torch.norm(tensor1)
    norm_tensor2 = torch.norm(tensor2)

    # Calculate the cosine similarity
    similarity = dot_product / (norm_tensor1 * norm_tensor2)

    return similarity

def cosine_similarity(sen1, sen2):
    assert len(sen1) == len(sen2), "Sentence lengths are not equal"

    res_vec = []
    for word in range(len(sen1)):
        res_vec.append(cosine_similarity_2_tensors(sen1[word], sen2[word]))
    return res_vec

In [ ]:
cosine_similarity_res = cosine_similarity(token_vecs_cat, token_vecs_cat2)
cosine_similarity_res

[tensor(0.9724),
 tensor(0.9487),
 tensor(0.9855),
 tensor(0.9867),
 tensor(0.9746),
 tensor(0.9765),
 tensor(0.9757),
 tensor(0.9840),
 tensor(0.9805),
 tensor(0.9894),
 tensor(0.9533),
 tensor(0.9677),
 tensor(0.9812),
 tensor(0.9790),
 tensor(0.9275),
 tensor(0.9729),
 tensor(0.9666),
 tensor(0.9850),
 tensor(0.9230),
 tensor(0.5345),
 tensor(0.9140),
 tensor(0.9237),
 tensor(0.6716),
 tensor(0.6131),
 tensor(0.5775),
 tensor(0.5866),
 tensor(0.5695),
 tensor(0.8508),
 tensor(0.9433),
 tensor(0.9849)]

## test cosine module similarity

In [ ]:
def cosine_module_2_tensors(tensor1, tensor2):
    cosine = cosine_similarity_2_tensors(tensor1, tensor2)
    module = 1 - torch.abs(torch.norm(tensor1) - torch.norm(tensor2))/(torch.norm(tensor1) + torch.norm(tensor2))

    cosine_module = 1/2 * (cosine + module)

    return cosine_module

def cosine_module(sen1, sen2):
    assert len(sen1) == len(sen2), "Sentence lengths are not equal"

    res_vec = []
    for word in range(len(sen1)):
        res_vec.append(cosine_module_2_tensors(sen1[word], sen2[word]))
    return res_vec

In [ ]:
cosine_module_res = cosine_module(token_vecs_cat, token_vecs_cat2)
cosine_module_res

[tensor(0.9729),
 tensor(0.9686),
 tensor(0.9924),
 tensor(0.9926),
 tensor(0.9818),
 tensor(0.9778),
 tensor(0.9816),
 tensor(0.9894),
 tensor(0.9820),
 tensor(0.9931),
 tensor(0.9692),
 tensor(0.9798),
 tensor(0.9903),
 tensor(0.9868),
 tensor(0.9493),
 tensor(0.9828),
 tensor(0.9830),
 tensor(0.9907),
 tensor(0.9526),
 tensor(0.6996),
 tensor(0.9545),
 tensor(0.9525),
 tensor(0.8316),
 tensor(0.7971),
 tensor(0.7316),
 tensor(0.7650),
 tensor(0.7807),
 tensor(0.9096),
 tensor(0.9647),
 tensor(0.9920)]

## test finetuned model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! git clone https://github.com/callmetandat/SRL-for-BioBERT.git

Cloning into 'SRL-for-BioBERT'...
remote: Enumerating objects: 1649, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 1649 (delta 58), reused 12 (delta 5), pack-reused 1558
Receiving objects: 100% (1649/1649), 32.74 MiB | 4.80 MiB/s, done.
Resolving deltas: 100% (1125/1125), done.
Updating files: 100% (416/416), done.


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 623.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=d563dfb5c45822e6ed392738b4acd6effc09d7b494215778dea63874929ca112
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
#making and loading model
import torch
import sys
import os
import logging
sys.path.insert(1, '/content/SRL-for-BioBERT/')
import matplotlib.pyplot as plt
from models.model import multiTaskModel
from transformers import BertTokenizer, BertModel, BertConfig
from utils.data_utils import TaskType, NLP_MODELS

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

In [ ]:
# dmis-lab/biobert-base-cased-v1.2
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2', do_lower_case=True,truncation=True)


In [ ]:
#text = "the a/t-rich mut sequence indicates that normal splicing was abolished by a g-to-a transition at the first nucleotide of intron 2."
text1 = "torc induction was abolished by deletion of the distal tor box (box1), which interrupt this open reading frame."
text2 = "torc induction was abolished by deletion of the distal tor box (box3), when demolish this writing frame."

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize("[CLS] " + text1 + " [SEP]")
tokenized_text2 = tokenizer.tokenize("[CLS] " + text2 + " [SEP]")

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
indexed_tokens2 = tokenizer.convert_tokens_to_ids(tokenized_text2)

segments_ids = [1] * len(tokenized_text)
segments_ids2 = [1] * len(tokenized_text2)

tokens_tensor = torch.tensor([indexed_tokens], device='cuda:0')
segments_tensors = torch.tensor([segments_ids], device='cuda:0')

tokens_tensor2 = torch.tensor([indexed_tokens2], device='cuda:0')
segments_tensors2 = torch.tensor([segments_ids2], device='cuda:0')

In [ ]:

# Load pre-trained model (weights)

loadedDict = torch.load('/content/drive/MyDrive/output_SRL_model/multi_task_model_8_367.pt', map_location=torch.device('cuda'))

taskParams = loadedDict['task_params']

modelName =taskParams.modelType.name.lower()
_, _ , tokenizerClass, defaultName = NLP_MODELS[modelName]
configName = taskParams.modelConfig

#making tokenizer for model
tokenizer = tokenizerClass.from_pretrained(configName)

allParams = {}
allParams['task_params'] = taskParams
allParams['gpu'] = torch.cuda.is_available()
# dummy values
allParams['num_train_steps'] = 10
allParams['warmup_steps'] = 0
allParams['learning_rate'] = 2e-5
allParams['epsilon'] = 1e-8


model = multiTaskModel(allParams)
model.load_multi_task_model(loadedDict)


attention_masks = torch.tensor([[1] * 28], device='cuda:0')
with torch.no_grad():

    output = model.network(tokens_tensor, segments_tensors, attention_masks, 0, 'conllsrl')

output[0].shape

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


torch.Size([28, 15])

In [ ]:
print(model)

In [ ]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))
print(token_vecs_cat)

In [ ]:
text = ["in addition, deletion of the distal tor box (box1) abolished torc induction whereas the presence of a dna fragment starting three bases upstream from box1 suffices for normal torc expression."]
import sys
sys.path.insert(1, '../')
from infer_pipeline import inferPipeline
from utils.data_utils import TaskType
pipe = inferPipeline(modelPath='./output/multi_task_model_8_367.pt', maxSeqLen=50)

data = pipe.infer(text, ['conllsrl'])

/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializin

taskName :  odict_values(['conllsrl'])
allTasksList :  [{'data_task_id': 0, 'data_': [{'uid': 0, 'label': [12, 2, 13, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 'token_id': [101, 1045, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'type_id': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}], 'data_task_type': <TaskType.NER: 3>, 'data_task_name': 'conllsrl'}]
inferDataLoader :  <torch.utils.data.dataloader.DataLoader object at 0x7fa01171aeb0>


Eval: 100%|██████████| 1/1 [00:00<00:00,  5.95it/s]


In [ ]:
import json
import torch

def read_data(readPath):

    with open(readPath, 'r', encoding = 'utf-8') as file:
        taskData = []
        for i, line in enumerate(file):
            sample = json.loads(line)
            taskData.append(sample)

    return taskData

In [ ]:
import os
import pickle
from models.model import multiTaskModel

def get_embedding_finetuned(dataDir, readFile, wrtDir):
    data = read_data(os.path.join(dataDir, readFile))

    # Load finetuned model
    loadedDict = torch.load('../output/multi_task_model_9_13050.pt', map_location=torch.device('cpu'))

    taskParams = loadedDict['task_params']

    allParams = {}
    allParams['task_params'] = taskParams
    allParams['gpu'] = torch.cuda.is_available()
    # dummy values
    allParams['num_train_steps'] = 10
    allParams['warmup_steps'] = 0
    allParams['learning_rate'] = 2e-5
    allParams['epsilon'] = 1e-8


    model = multiTaskModel(allParams)
    model.load_multi_task_model(loadedDict)

    for i, line in enumerate(data):
        tokens_id = line['token_id']
        segments_id = line['type_id']
        u_id = line['uid']
        attention_mask = line['mask']

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([tokens_id])

        segments_tensors = torch.tensor([segments_id])

        print("Processed {} rows...".format(i))
        with torch.no_grad():
            outputs = model.network(tokens_tensor, segments_tensors, attention_mask, 0, 'conllsrl')
            print("")
            hidden_states = outputs[1][2]
            print("done {} rows...".format(i))
        # `hidden_states` is a Python list.
        # Each layer in the list is a torch tensor.
        # `token_vecs` is a tensor with shape [50 x 768]

        ## WORD EMBEDDING
        token_embeddings = torch.stack(hidden_states, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)

        token_embeddings.size()

        # Stores the token vectors, with shape [22 x 3,072]
        token_vecs_cat = []

        # `token_embeddings` is a [22 x 12 x 768] tensor.

        # For each token in the sentence...
        for token in token_embeddings:
            # `token` is a [12 x 768] tensor

            # Concatenate the vectors (that is, append them together) from the last four layers.
            # Each layer vector is 768 values, so `cat_vec` is length 3,072.
            cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

            # Use `cat_vec` to represent `token`.
            token_vecs_cat.append(cat_vec)

        features = {
                'uid': u_id,
                'vec': token_vecs_cat}
        # write u_id and token_vecs_cat to file
        with open(os.path.join(wrtDir, 'finetuned_vecs_{}.pkl'.format(readFile.split('.')[0])), 'wb') as vecs_wri:
            pickle.dump(features, vecs_wri)

In [ ]:
dataDir = '/mnt/c/Users/Phat Pham/Documents/THESIS/SRL-for-BioBERT/data/coNLL_tsv/bert-base-uncased_prepared_data/'
readFile = 'ner_conll_testa_abolish.json'
wriDir = 'test_get_embedding_finetuned'
vec_finetuned = get_embedding_finetuned(dataDir, readFile, wriDir)

/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


: 

# calculate cosine similarity

In [ ]:
import os
import pickle
def read_json_file(dirPath, wrtDir):
    files = []
    for path in os.listdir(dirPath):
        if os.path.isfile(os.path.join(dirPath, path)):
            files.append(path)


    for file in files:
        with open(os.path.join(dirPath, file), 'rb') as f:
            data = pickle.load(f)

        vec_origin = get_embedding(data)
        vec_finetuned = get_embedding_finetuned(data)
        cosine = cosine_similarity(vec_origin, vec_finetuned)
        cosine_module = cosine_module(vec_origin, vec_finetuned)

        features = {'uid': data['uid'], 'cosine': cosine, 'cosine_module': cosine_module}
        with open(os.path.join(wrtDir, 'vecs_{}.pkl'.format(file.split('.')[0])), 'wb') as vecs_wri:
            pickle.dump(features, vecs_wri)
    return data

data = read_json_file('./word_vecs_output/')
data

In [ ]:
from transformers import BertConfig, BertModel, BertTokenizer

data = read_data('/mnt/c/Users/Phat Pham/Documents/THESIS/SRL-for-BioBERT/data/coNLL_tsv/bert-base-uncased_prepared_data/ner_conll_testa_abolish.json')

print("data shape: ", len(data))  # 280
tokens_id = data[0]['token_id']
segments_id = data[0]['type_id']


# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([tokens_id])
segments_tensors = torch.tensor([segments_id])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('dmis-lab/biobert-base-cased-v1.2',
                                output_hidden_states = True # Whether the model returns all hidden-states.
                                )
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)
    print("outputs: ", len(outputs)) # 2
    hidden_states = outputs[2]


print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")  #13
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i])) #1
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i])) #50
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))     # 768


# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states)) # <class 'tuple'>

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size()) # torch.Size([1, 50, 768])


## Sentence Vectors
# `hidden_states` has shape [13 x 1 x 50 x 768]

# `token_vecs` is a tensor with shape [50 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
print ("Our final sentence embedding vector of shape:", sentence_embedding.size()) # torch.Size([768])

data shape:  280
outputs:  3
Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 50
Number of hidden units: 768
      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 50, 768])
Our final sentence embedding vector of shape: torch.Size([768])


In [ ]:
import pickle
with open('models/outputs.pkl', 'rb') as f:
    token_vecs_cat = pickle.load(f)

print("token_vecs_cat: ", len(token_vecs_cat)) # 50
print(token_vecs_cat['uid']) # (3072,)


token_vecs_cat:  2
['22110', '22111', '22112', '22113', '22114', '22115', '22116', '22117', '22118', '22119', '22120', '22121', '22122', '22123', '22124', '22125', '22126', '22127', '22128', '22129', '22130', '22131', '22132', '22133', '22134', '22135', '22136', '22137', '22138', '22139', '22140', '22141']


In [ ]:
import torch.nn as nn
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
print(output.backward())

# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
print(output.shape)
print(output.backward())

# read pickle file for test word present in the model

In [ ]:
import torch
import pickle
data_file = './vecs_ner_conll_testa_alter.pkl'
with open(data_file, 'rb') as f:
    data = pickle.load(f)

data

In [ ]:
import pandas as pd


data_file = './vecs_ner_conll_testa_abolish.pkl'
with open(data_file, 'rb') as f:
    data = pickle.load(f)

data['vec']
df = pd.DataFrame(data['vec'])

In [ ]:
df.shape

(50, 3072)

# test get word embedding from biobert

In [ ]:
import json
file = './data/coNLL_tsv/dmis-lab/biobert-base-cased-v1.2_prepared_data/ner_conll_testa_abolish.json'
def read_data(readPath):

    with open(readPath, 'r', encoding = 'utf-8') as file:
        taskData = []
        for i, line in enumerate(file):
            sample = json.loads(line)
            taskData.append(sample)

    return taskData
data = read_data(file)



In [ ]:
from transformers import BertModel, BertTokenizer
bertmodel = BertModel.from_pretrained('dmis-lab/biobert-base-cased-v1.2', output_hidden_states =True)

def get_embedding(line):
    '''
    get word presentation from origin biobert model
    input:json file in coNLL_tsv/dmis-lab/biobert-base-cased-v1.2_prepared_data
    output: word presentation with shape for each word [3072]
    '''
    tokens_id = line['token_id']
    segments_id = line['type_id']

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([tokens_id])

    segments_tensors = torch.tensor([segments_id])

    with torch.no_grad():
        outputs = bertmodel(tokens_tensor, segments_tensors)

        hidden_states = outputs[2]

    ## WORD EMBEDDING
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:                  # `token` is a [12 x 768] tensor

        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0) # last four layers

        token_vecs_cat.append(cat_vec) #cat_vec is 3072
    return token_vecs_cat

# test get word embedding from finetuned biobert model

In [ ]:
# Load finetuned model
loadedDict = torch.load('./output/multi_task_model_0_1305.pt') #, map_location=torch.device('cpu'))

taskParams = loadedDict['task_params']

allParams = {}
allParams['task_params'] = taskParams
allParams['gpu'] = torch.cuda.is_available()
# dummy values
allParams['num_train_steps'] = 10
allParams['warmup_steps'] = 0
allParams['learning_rate'] = 2e-5
allParams['epsilon'] = 1e-8

finetuned_model = multiTaskModel(allParams)
finetuned_model.load_multi_task_model(loadedDict)


/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

finetuned_model._to_cuda(device)

AttributeError: 'torch.device' object has no attribute 'cuda'

: 

In [ ]:
from models.model import multiTaskModel
import torch
def get_embedding_finetuned(finetuned_model, line):

    tokens_id = line['token_id']
    segments_id = line['type_id']

    attention_mask = torch.tensor([line['mask']])

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([tokens_id])

    segments_tensors = torch.tensor([segments_id])

    with torch.no_grad():
        outputs = finetuned_model.network(tokens_tensor, segments_tensors, attention_mask, 0, 'conllsrl')
        print(outputs.shape)   # torch.Size([1, 50, 15])
        print(outputs)
        hidden_states = outputs[0][2]

    ## WORD EMBEDDING
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat = []

    # For each token in the sentence...
    for token in token_embeddings:

        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat.append(cat_vec)

    return token_vecs_cat

In [ ]:
word_vec = get_embedding_finetuned(data[0])

/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


torch.Size([1, 50, 15])
tensor([[[-1.1846e+00, -6.8459e-01,  9.8238e-01,  6.8634e-02, -9.1011e-02,
          -1.5405e+00, -7.6124e-01, -1.1030e+00, -8.5586e-02, -1.5470e+00,
          -1.0949e+00,  1.5951e-01,  8.3361e+00,  9.6597e-01, -7.1799e-01],
         [ 3.1445e+00, -2.4696e-01,  4.4184e+00, -3.0287e+00,  3.9902e+00,
           8.4581e-01, -1.1889e+00, -2.3679e+00, -1.2814e+00,  1.6745e+00,
          -1.1275e+00, -1.0425e+00, -2.8933e+00, -2.6966e+00, -1.9981e+00],
         [ 2.4267e+00,  1.9791e+00,  2.6731e+00, -3.5098e+00,  9.7528e-01,
           1.9944e+00, -1.3995e+00, -2.4866e+00, -1.3075e+00,  7.7378e-01,
           9.7236e-01, -1.4114e+00, -3.1437e+00, -2.9934e+00,  9.4210e-01],
         [-1.1750e+00, -6.8287e-01,  8.9483e-01, -1.6155e+00, -7.0446e-01,
          -5.0873e-01, -7.1245e-01, -1.3488e+00, -3.1651e-01, -7.8965e-01,
          -1.1418e-01, -1.0203e+00,  7.1724e-02, -1.0655e+00,  8.2317e+00],
         [ 1.0061e+00,  2.3499e+00,  7.0568e-01, -3.1166e+00,  4.0577e-0

NameError: name 'token_vecs_cat' is not defined

In [ ]:
representations = {}
for i, line in enumerate(data):
    word_vec = get_embedding_finetuned(line)
    representations[i] = {'uid': line['uid'], 'vec': word_vec}

In [ ]:
len(representations)

280